In [2]:
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached sqlalchemy-2.0.43-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached aiohttp-3.12.15-cp311-cp311-win_amd64.whl.metadata (7.9 kB)
  Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.7.0-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached multidict-6.6.4-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached propcache-0.3.2-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp311-cp311-win_amd64.whl.metadata (76 kB)
  Using cac


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


True

### 임베딩

In [ ]:
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
index_name = 'tax-index10'
vectorstore = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embedding,
) #DB로부터 인덱스값을 가져온다.// KEY 필요없다.

llm = ChatOpenAI(model='gpt-5')
prompt =hub.pull("rlm/rag-prompt")
retriever = vectorstore.as_retriever(search_kwargs={'k':3})

qa_chain =RetrievalQA.from_chain_type(
    llm= llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
)

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt2 = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전 : {dictionary}

    질문: {{question}}
""")

dictionary_chain = prompt2 | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain  # 2개의 체인연결 한것이 tax_chain// # query 쓰는 이유 == qa_chain



In [36]:
#query = "연봉 5천만원 거주자의 종합소득세는 얼마인가?"

query = "연봉 5천만원 직장인 소득세는 얼마인가?"

In [ ]:

retriever.invoke(query)

[Document(id='27de31fa-0d05-4d74-96ac-30e12cd04211', metadata={'source': './data/tax.docx'}, page_content='| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |'),
 Document(id='d101f46d-6637-48dd-b49c-d470b75359db', metadata={'source': './data/tax.docx'}, page_content='| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|'),
 Document(id='de00d514-9442-428d-a77c-2880acb81d8d', metadata={'source': './data/tax.docx'}, page_content='3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는

In [42]:
ai_message = qa_chain.invoke({"query": query})

In [43]:
ai_message

{'query': '연봉 5천만원 직장인 소득세는 얼마인가?',
 'result': '과세표준이 5,000만원일 때 종합소득세 산출세액은 84만원 + (5,000만원−1,400만원)×15% = 624만원입니다. 다만 “연봉” 5,000만원의 실제 과세표준은 각종 공제 후 결정되므로 실제 세액은 달라질 수 있습니다.'}

In [ ]:
# 테스트 용
# new_question = dictionary_chain.invoke({"question": query})
# new_question

'연봉 5천만원인 거주자의 소득세는 얼마인가?'

In [47]:
ai_message = tax_chain.invoke({"question": query})
ai_message

{'query': '연봉 5천만원 거주자 소득세는 얼마인가?',
 'result': '과세표준이 5천만원이면 종합소득 산출세액은 624만원입니다. 근로소득자 표준세액공제(13만원)만 적용하면 결정세액은 약 611만원입니다. 다만 “연봉 5천만원”의 실제 과세표준은 각종 공제 후 금액이므로 실제 세액은 달라질 수 있습니다.'}